# Digital Elevation Model (DEM)

This notebook processes a Digital Elevation Model (DEM) for the Limmat Valley. The DEM provides ground surface elevation data, which is crucial for setting up the top of our groundwater model.

## Data Source

We will use the DHM25 dataset from Swisstopo, which has a grid resolution of 25 meters. This resolution provides a good balance between detail and computational efficiency for our model.

**Action Required:**

1.  **Download the data:** Visit the [Swisstopo website](https://www.swisstopo.admin.ch/en/height-model-dhm25) (accessed July 4, 2025) to download the DHM25 dataset.
2.  **Set up your data folder:** Unzip the downloaded file and place the `ASCII_GRID_1part` folder into a directory structure like this: `~/applied_groundwater_modelling_data/limmat/gis/DHM25/`.

If you use a different folder structure, you will need to update the file paths in the code cells below.

The following sections will guide you through reading, processing, and visualizing the DEM data.

In [ ]:
# Import required libraries
import os
import pickle
import rasterio
import numpy as np
from rasterio.transform import from_origin
from rasterio.mask import mask
from shapely.geometry import box, mapping
from geopandas import GeoDataFrame
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import plotly.graph_objects as go

## Reading the Elevation Data

The DEM data comes in an ASCII grid format. We need to read this file to get the elevation data and its spatial reference.

The Python function below, `read_elevation_data`, is designed to:

1.  Read the header of the ASCII file to understand the grid's properties (dimensions, coordinates, etc.).
2.  Load the elevation data into a NumPy array.
3.  Use a `.pkl` file to cache the data. This is a great time-saver! After you run this notebook once, the processed data will be saved. The next time you run it, the data will be loaded from the cache, which is much faster than reading the original ASCII file again.
4.  Handle potential issues, with a fallback mechanism using the `rasterio` library if the primary method fails.


In [ ]:
def read_elevation_data(input_path=None, pickle_path=None):
    """
    Read elevation data from an ASCII grid file, assuming it's in the LV03 coordinate system.

    This function includes caching to speed up subsequent runs.

    Parameters:
    -----------
    input_path : str, optional
        Path to the ASCII grid file. If None, a default path is used.
    pickle_path : str
        Path to the pickle file for caching. If None, uses default path.
    
    Returns:
    --------
    tuple
        (elevation_data, transform)
    """
    if input_path is None:
        # Default path to the ASCII grid file
        input_path = os.path.expanduser('~/applied_groundwater_modelling_data/limmat/gis/DHM25/ASCII_GRID_1part/dhm25_grid_raster.asc')
    if pickle_path is None:
        # Default path to the pickle file for caching
        pickle_path = os.path.expanduser('~/applied_groundwater_modelling_data/limmat/gis/DHM25/elevation_data.pkl')
    
    # Create directory for pickle file if it doesn't exist
    pickle_dir = os.path.dirname(pickle_path)
    os.makedirs(pickle_dir, exist_ok=True)

    # Check for cached data
    if os.path.exists(pickle_path):
        print('Reading from cache...')
        try:
            with open(pickle_path, 'rb') as f:
                data = pickle.load(f)
                if isinstance(data, tuple) and len(data) == 2:
                    elevation_data, transform = data
                    return elevation_data, transform
                else:
                    print("Old cache format detected. Deleting and re-reading from source.")
                    os.remove(pickle_path)
        except (pickle.UnpicklingError, ValueError, EOFError) as e:
            print(f"Error reading cache file: {e}. Deleting and re-reading from source.")
            os.remove(pickle_path)

    # If cache doesn't exist or was invalid, read from source
    print('Reading from source...')
    try:
        # Read ASCII grid file headers manually to get proper coordinates
        with open(input_path, 'r') as f:
            headers = {}
            for _ in range(6):  # Standard ESRI ASCII grid has 6 header lines
                line = f.readline().strip().split()
                headers[line[0].lower()] = float(line[1])

        # Extract key parameters from the header
        ncols = int(headers['ncols'])
        nrows = int(headers['nrows'])
        xllcorner = headers['xllcorner']
        yllcorner = headers['yllcorner']
        cellsize = headers['cellsize']
        nodata = headers.get('nodata_value', -9999)

        print("ASCII grid parameters:")
        print(f"  Columns: {ncols}, Rows: {nrows}")
        print(f"  Lower-left X: {xllcorner}, Lower-left Y: {yllcorner}")
        print(f"  Cell size: {cellsize}")

        # Load the elevation data, skipping the header
        elevation_data = np.loadtxt(input_path, skiprows=6)
        # Replace nodata values with NaN
        elevation_data = np.where(elevation_data == nodata, np.nan, elevation_data)
        
        # Create the proper transform
        transform = from_origin(xllcorner, yllcorner + nrows * cellsize, cellsize, cellsize)

        # Save to cache
        with open(pickle_path, 'wb') as f:
            pickle.dump((elevation_data, transform), f)
        
        return elevation_data, transform
        
    except Exception as e:
        print(f"Error reading ASCII grid manually: {e}")
        # If the manual method fails, try using rasterio as a fallback
        try:
            print("Attempting to read with rasterio...")
            with rasterio.open(input_path) as dataset:
                print(f"CRS from rasterio: {dataset.crs}")
                elevation_data = dataset.read(1)
                transform = dataset.transform
                with open(pickle_path, 'wb') as f:
                    pickle.dump((elevation_data, transform), f)
                
                return elevation_data, transform
        except Exception as e2:
            print(f"Error with rasterio fallback: {e2}")
            raise RuntimeError("Could not read the elevation data")


In [ ]:
# Read the DEM data. If you've run this before, it will load from the cache.
dem, transform = read_elevation_data()

# Display information about the loaded DEM
print("\nDEM Information:")
print("DEM shape:", dem.shape)
print("Transform matrix properties:")
print(f"  Pixel width (x resolution): {transform.a} m")
print(f"  Pixel height (y resolution): {abs(transform.e)} m")
print(f"  Upper left x: {transform.c} m")
print(f"  Upper left y: {transform.f} m")

## Getting Elevation at a Specific Location

Once the DEM is loaded, you can retrieve the elevation for any coordinate within the model area. This is useful for checking specific points or for interpolating data.

The functions below help convert from LV03 map coordinates (Easting, Northing) to the raster's internal row and column indices, allowing us to look up the elevation value.

In [ ]:
def lv03_to_rowcol(transform, east, north):
    """
    Convert LV03 coordinates to row/column indices in the raster.
    """
    col, row = ~transform * (east, north)
    return int(row), int(col)

def get_elevation(model, east, north):
    """
    Get the elevation at a given LV03 coordinate.
    """
    data, transform = model
    row, col = lv03_to_rowcol(transform, east, north)

    if 0 <= row < data.shape[0] and 0 <= col < data.shape[1]:
        return data[row, col]
    else:
        raise ValueError(f"Coordinates ({east}, {north}) are outside the DEM bounds")

In [ ]:
# Example: Get the elevation for Zurich Main Station (Hauptbahnhof)
zurich_hb_east = 683500
zurich_hb_north = 246700

try:
    elevation = get_elevation((dem, transform), zurich_hb_east, zurich_hb_north)
    print(f"Elevation at Zurich HB ({zurich_hb_east}, {zurich_hb_north}): {elevation:.2f} meters")
except ValueError as e:
    print(f"Error: {e}")

## Saving the Processed DEM as a GeoTIFF

The original ASCII format is not always the most efficient for GIS applications. The GeoTIFF format is a standard for raster data because it can store the data and its coordinate system information in a single file.

Saving our processed DEM as a GeoTIFF makes it easier to use in other GIS software (like QGIS) or in later modeling steps. We will explicitly set the Coordinate Reference System (CRS) to LV03 (EPSG:21781).

In [ ]:
# Define the output path for the GeoTIFF file
output_tiff = os.path.expanduser('~/applied_groundwater_modelling_data/limmat/gis/DHM25/dhm25_grid_raster_lv03.tif')
# Check if the file already exists to avoid rewriting it
if not os.path.exists(output_tiff):
    print(f"\nSaving DEM with explicit LV03 CRS to {output_tiff}")
    with rasterio.open(
        output_tiff,
        'w',
        driver='GTiff',
        height=dem.shape[0],
        width=dem.shape[1],
        count=1,
        dtype=dem.dtype,
        crs='EPSG:21781',  # Explicitly set LV03 CRS
        transform=transform,
        nodata=np.nan
    ) as dst:
        dst.write(dem, 1)
    print("GeoTIFF saved successfully.")
else:
    print("GeoTIFF file already exists. Skipping save.")

## Visualizing the DEM

A great way to understand the DEM data is to create a plot. The code below will generate a map of the elevations.

Some parts of the raw data may contain negative elevation values, which are likely artifacts or represent areas below sea level that are not relevant for our study area. For clarity, we will mask out these negative values and any `NaN` values in our plot.

In [ ]:
# Basic Visualization

# Create a copy of the DEM and mask out negative values and NaNs
dem_positive = dem.copy()
masked_dem = np.ma.masked_where((dem_positive <= 0) | np.isnan(dem_positive), dem_positive)

plt.figure(figsize=(10, 8))
plt.imshow(masked_dem, cmap='terrain', vmin=0)
plt.colorbar(label='Elevation (m)')
plt.title('Digital Elevation Model (DEM) - Positive Elevations Only')
plt.xlabel('Column Index')
plt.ylabel('Row Index')
plt.tight_layout()
plt.show()

## Clipping the DEM to the Area of Interest

For our case study, we are only interested in a specific portion of the Limmat Valley. Working with the full DEM is computationally expensive and includes data far outside our model domain. Therefore, we will "clip" the DEM to a smaller bounding box that covers our area of interest.

### Step 1: Define the Bounding Box and Convert Coordinates

The area of interest is often provided in a specific coordinate system. Here, we have the coordinates in **LV95** (the newer Swiss system), but our DEM is in **LV03**. The first step is to define the bounding box in LV95 and then convert it to LV03.

*   **LV95 to LV03 Conversion:** A simple transformation is used: `X_LV03 = X_LV95 - 2,000,000` and `Y_LV03 = Y_LV95 - 1,000,000`.

In [ ]:
# Define the bounding box coordinates of the area of interest in LV95
lv95_coords = {
    'min_x': 2672834,  # Left
    'max_x': 2684405,  # Right
    'min_y': 1246174,  # Bottom
    'max_y': 1253235   # Top
}

# Convert from LV95 to LV03 for compatibility with our DEM
lv03_coords = {
    'min_x': lv95_coords['min_x'] - 2000000,
    'max_x': lv95_coords['max_x'] - 2000000,
    'min_y': lv95_coords['min_y'] - 1000000,
    'max_y': lv95_coords['max_y'] - 1000000
}

print("Bounding Box (LV03):")
print(f"  ({lv03_coords['min_x']}, {lv03_coords['min_y']}) to ({lv03_coords['max_x']}, {lv03_coords['max_y']})")

# Create a geometry object for the bounding box
bbox_geom = box(
    lv03_coords['min_x'], lv03_coords['min_y'],
    lv03_coords['max_x'], lv03_coords['max_y']
)

# Create a GeoDataFrame, which is useful for plotting and spatial operations
bbox_gdf = GeoDataFrame({'geometry': [bbox_geom]}, crs='EPSG:21781')  # LV03 CRS

### Step 2: Perform the Clipping

Now that we have the bounding box in the correct coordinate system, we can use the `rasterio` library to clip the original DEM. The `mask` function will cut out the portion of the raster that falls within our bounding box geometry.

In [ ]:
# Path to the full DEM GeoTIFF we saved earlier
dem_tiff_path = '~/applied_groundwater_modelling_data/limmat/gis/DHM25/tiff_clipped.tif'

print(f"Clipping DEM to the defined bounding box...")

with rasterio.open(dem_tiff_path) as src:
    # Use the mask function to clip the raster
    clipped_dem, clipped_transform = mask(src, [mapping(bbox_geom)], crop=True)
    clipped_dem = clipped_dem[0]  # The result is a 3D array, we only need the first band

print("✅ DEM clipping successful!")
print(f"Original DEM shape: {dem.shape}")
print(f"Clipped DEM shape: {clipped_dem.shape}")

### Step 3: Visualize the Clipped DEM

It's always a good practice to visualize the result to ensure the clipping operation worked as expected. The plot below shows the clipped area with a hillshade effect and the red bounding box outline.

In [ ]:
# Process the clipped DEM for visualization (masking negative values)
clipped_dem_positive = np.ma.masked_where((clipped_dem <= 0) | np.isnan(clipped_dem), clipped_dem)

# Create hillshade for the clipped DEM
hillshade_clipped = np.nan_to_num(clipped_dem, nan=0.0)
hillshade_clipped[hillshade_clipped < 0] = 0
ls = LightSource(azdeg=315, altdeg=45)
hillshade = ls.hillshade(hillshade_clipped, vert_exag=3)

# Get the extent for plotting
x_min, y_max = clipped_transform.c, clipped_transform.f
x_max = x_min + clipped_transform.a * clipped_dem.shape[1]
y_min = y_max - abs(clipped_transform.e) * clipped_dem.shape[0]
extent = [x_min, x_max, y_min, y_max]

# Create the plot
fig, ax = plt.subplots(figsize=(12, 10))
ax.imshow(hillshade, cmap='gray', extent=extent, alpha=0.5)
dem_plot = ax.imshow(clipped_dem_positive, cmap='terrain', extent=extent, alpha=0.7)
bbox_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2, label='Bounding Box')

cbar = plt.colorbar(dem_plot, ax=ax)
cbar.set_label('Elevation (m)')
ax.legend()
plt.title('Clipped DEM with Hillshade', fontsize=16)
plt.xlabel('Easting (m, LV03)', fontsize=12)
plt.ylabel('Northing (m, LV03)', fontsize=12)
plt.tight_layout()
plt.show()

### Step 4: Save the Clipped DEM

Saving the smaller, clipped DEM to a new GeoTIFF file is efficient for future work. This file can be easily shared or loaded into other modeling or GIS software without needing to re-process the large original file.

In [ ]:
# Define the output path for the clipped GeoTIFF
output_clipped_tiff = '~/applied_groundwater_modelling_data/limmat/gis/DHM25/DEM_clipped_bbox.tif'

with rasterio.open(
    output_clipped_tiff,
    'w',
    driver='GTiff',
    height=clipped_dem.shape[0],
    width=clipped_dem.shape[1],
    count=1,
    dtype=clipped_dem.dtype,
    crs='EPSG:21781',  # LV03
    transform=clipped_transform,
    nodata=np.nan
) as dst:
    dst.write(clipped_dem, 1)

print(f"✅ Clipped DEM saved to: {output_clipped_tiff}")

## Interactive Zoomable Map of the Clipped Area

Now for the main goal: creating an interactive map to inspect the details of our clipped study area. We will use the Plotly library, which creates web-based, zoomable charts.

This is particularly useful for identifying and examining specific features or anomalies in the elevation data, like the strange shapes you may have noticed in the Limmat Valley.

In [ ]:
# Install plotly if not already installed
import sys
!{sys.executable} -m pip install plotly

### Ensuring Interactivity

To ensure Plotly generates a zoomable map within the notebook, we explicitly set the default renderer.

In [ ]:
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
import plotly.graph_objects as go

# Use the clipped and masked DEM from the previous steps
fig = go.Figure(data=go.Heatmap(
    z=clipped_dem_positive,
    colorscale='earth',
    zmin=0,
    colorbar=dict(title='Elevation (m)')
))

fig.update_layout(
    title='Interactive DEM of the Limmat Valley (Clipped Area)',
    xaxis_title='Easting (m)',
    yaxis_title='Northing (m)',
    yaxis_autorange='reversed' # Match the orientation of the previous plots
)

fig.show()